### Dependencies

In [1]:
import sys
sys.path.append('/home/jovyan/farsite-devAPI/src/')
sys.path.append('/home/jovyan/python-helper/src/')

import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import MultiPolygon, Polygon

import uuid
import random

import farsiteutils_v2 as futils
from kalmanutils import calculate_uncertainties_observed, interpolate_perimeter
from kalmanutils import State, sample_geometry, calculate_max_area_geom, calculate
from kalmanutils import calculate_rms, calculate_area_diff, align_vertices, interpolate_geometries
from kalmanutils import update_EnKF, fill_zeros, create_ensemble_matrix

from loggers import TimeEstimator

from matplotlib import pyplot as plt

import datetime
import pickle

### Default Values

In [2]:
SELECTED_FIRE_DEFAULT = 'Maria2019'
DATA_PATH = '/home/jovyan/data/'    # Folder to keep all the data while running the simulations

### Select a fire

```selected_fire``` variable holds the name of the fire selected. ```selected_fire = 'Maria2019'``` for now

In [3]:
fp = futils.FilePaths(DATA_PATH)
usr = futils.User(fp)

unique_desc = usr.db.gdfignitionAll['description'].unique()
print(f'Available fires are {unique_desc}')

selected_fire = SELECTED_FIRE_DEFAULT
print(f'{selected_fire} is selected.\n')
if (unique_desc == selected_fire).any():
    print(f'{selected_fire} is found!')
else:
    raise ValueError(f'{selected_fire} not found in possible fire descriptions: {unique_desc}')

Database interaction not yet implemented. Use pickle file for dataframes instead!
Available fires are ['Maria2019' 'River2021' 'Bridge2021']
Maria2019 is selected.

Maria2019 is found!


### Selecting a reference observation
Reference observation is used to test the EnKF updates. It is in place of the actual observations. For now, all the references are a 60-minute FARSITE simulations at 10mph wind speed and wind moving directly from EAST. There are a total of 24 such calculations from the first point until the end.

In [28]:
objectidlst = [21231, 21232, 21234, 21235, 21236]
wdmaria = [37, 48, 48, 44, 16]
wsmaria = [11, 18, 18, 25, 17]

gdfmaria = gpd.read_file('/home/jovyan/farsite-devAPI/data/maria-wgs84/maria-wgs84.shp')
# gdfmaria[gdfmaria['objectid'].isin(objectidlst)][['objectid', 'YYYYMMDD', 'windspeed', 'winddirec']]

gdfobserved = usr.db.gdfignition[usr.db.gdfignition['objectid'].isin(objectidlst)]
# gdfobserved

In [ ]:
# # referenceidx_lst = [usr.db.gdfignition.index[0]] + usr.db.gdfignition[usr.db.gdfignition['objectid'] == '21231_RefTruth'].index.tolist()

# # 1 hour updates
# referenceidx_lst = [usr.db.gdfignition.index[0]] + [uuid.uuid4().hex for i in range(24)]
# for i in range(len(referenceidx_lst[:-1])):
#     print('\n---------------------------------')
#     print(f'Calculating {i}/{len(referenceidx_lst)}')
#     calculate(referenceidx_lst[i], referenceidx_lst[i+1], usr, label='21231_ws10wd90', windspeed = 10, winddirection = 90, dt=datetime.timedelta(minutes=30))

In [ ]:
# def plot_geometry(geom, ax, **kwargs):
#     if isinstance(geom, MultiPolygon):
#         for g in geom.geoms:
#             x,y = g.exterior.coords.xy
#             ax.plot(x,y, **kwargs)
#     else:
#         x,y = geom.exterior.coords.xy
#         ax.plot(x,y, **kwargs)

# def read_geom(gdfrow):
#     return gpd.read_file(gdfrow['filepath'])['geometry'][0]
        
# gdfreference = usr.db.gdfignition[usr.db.gdfignition['objectid']=='21231_ws10wd90']   # Reference true perimeters

# fig, ax = plt.subplots(1,1, dpi=200)

# # plot_geometry(read_geom(gdfreference.iloc[0]), ax=ax)
# # # plot_geometry(gdfreference['geometry'].iloc[-1], ax=ax)
# # # plot_geometry(gdfreference['geometry'].iloc[-1], ax=ax)
# # plot_geometry(read_geom(gdfreference.loc['dc8d1ae71a1f4f2c98fc927cfd2541f0']), ax=ax, color='black')
# # gpd.read_file(gdfreference.loc['9a532abc307a4296a93c6d8e08be909a']['filepath']).plot(ax=ax, color='black')

# for geom in gdfreference['geometry']:
#     plot_geometry(geom, ax=ax)
    
# ax.set_aspect('equal')

### Calculating the deviations

The deviations are calculated with the same parameters as the reference, except the first ignition is deviated and written at ```objectid=21231_RefTruth``` value. Similar to the reference observations, 12 consecutive FARSITE simulations were calculated with wd=90, ws=10 at 30-minute time intervals.

*Note* The variable ```usr``` now inludes all the reference and deviated observations in its ignition table

In [ ]:
# deviateidx_lst = [usr.db.gdfignition.index[0]] + [uuid.uuid4().hex for i in range(24)]
# for i in range(len(deviateidx_lst[:-1])):
#     print('\n---------------------------------')
#     print(f'Calculating {i}/{len(deviateidx_lst)}')
#     calculate(deviateidx_lst[i], deviateidx_lst[i+1], usr, label='21231_Deviate', windspeed = 10, winddirection = 60, dt=datetime.timedelta(minutes=30))

In [ ]:
referenceidx_lst

In [ ]:
deviateidx_lst

In [ ]:
gdfreference = usr.db.gdfignition.loc[referenceidx_lst]   # Reference true perimeters
gdfdeviate = usr.db.gdfignition.loc[deviateidx_lst]     # Same with reference calculation, starting from a deviated geometry

In [ ]:
fig, ax = plt.subplots(1,1, dpi=400)

plot_geometry(gdfreference.iloc[0]['geometry'], ax=ax)
plot_geometry(gdfdeviate.iloc[0]['geometry'], ax=ax)
plot_geometry(gdfreference.iloc[10]['geometry'], ax=ax)
plot_geometry(gdfdeviate.iloc[10]['geometry'], ax=ax)

### Starting calculation - 0 $\rightarrow$ 1

In [ ]:
# ws = 10
# wd = 60
nsamples = 1000
vertex_count = 100

data = {'calculations': [],
        'parameters': {'nsamples': nsamples,
                       'vertex_count': vertex_count},
       }

In [ ]:
%%time

for upix in range(1,len(gdfobserved)):
    print('------------------------------------')
    print(f'Calculating {upix-1}-->{upix}')
    
    # Generate samples of (ws, wd)
    wsar = np.random.normal(wsmaria[upix-1], np.sqrt(ws), nsamples)
    wdar = np.random.normal(wdmaria[upix-1], 15, nsamples)

    compareidx = {}
    # Run for the first ensemble 0->1

    usr_tmp = futils.User(fp)
    if upix == 1:
        igniteidx = gdfdeviate.index[upix-1]
    else:
        igniteidx = updateidx
        usr_tmp.db.gdfignition.loc[igniteidx] = usr.db.gdfignition.loc[igniteidx]
        
    referenceidx = gdfreference.index[upix]
    compareidx[igniteidx] = []
    
    time_estimator = TimeEstimator(len(wsar))
    for i, (wsflt, wdflt) in enumerate(zip(wsar, wdar)):
        wsint = round(wsflt)
        wdint = np.fmod(round(wdflt)+360, 360)

        print(time_estimator.info_str(i))
        # print(f'\nCalculating for wd={wdint}, ws={wsint}')

        # calculate compareidx for the updated
        compareidx[igniteidx].append(uuid.uuid4().hex)
        calculate(igniteidx, compareidx[igniteidx][-1], usr_tmp, label='21231_Deviated_1', windspeed = wsint, winddirection = wdint)    


    # Calculate the ensemble matrix for the deviated simulations
    Xt, aligned_geom, vertex_count = create_ensemble_matrix(usr_tmp.db.gdfsimulation, nsamples=nsamples, vertex_count=vertex_count)

    # Calculate ensemble matrix for the observations
    geom = gdfreference.loc[referenceidx, 'geometry']

    uncertainties = np.zeros(nsamples) + 100
    interpolate_geom = interpolate_geometries([geom], vertex_count)[0]
    aligned_vertices = align_vertices([aligned_geom, interpolate_geom])[1]
    st = State(Polygon(aligned_vertices))

    aligned_geoms_lst = []
    for i in range(nsamples):
        newgeom = sample_geometry(st, uncertainties).buffer(0)
        interpolate_geom = interpolate_geometries([newgeom], vertex_count)[0]
        aligned_vertices = align_vertices([aligned_geom, interpolate_geom])[1]
        aligned_geoms_lst.append(Polygon(aligned_vertices))

    Y, aligned_geom, vertex_count = create_ensemble_matrix(gpd.GeoDataFrame(geometry=aligned_geoms_lst), nsamples, vertex_count, aligned_geom, observed=True)


    X = update_EnKF(Xt, Y, aligned_geom)
    ymean = X[1:-2:2,:].mean(axis=1)
    xmean = X[:-2:2,:].mean(axis=1)

    Xgeom = Polygon(zip(xmean, ymean))

    updateidx = uuid.uuid4().hex
    filepath = f'/home/jovyan/data/ignitions/Maria{updateidx}.shp'
    gpd.GeoDataFrame({'FID': [0], 'geometry':Xgeom}, 
                 crs='EPSG:5070').to_file(filepath)

    gdfupdate = gpd.GeoDataFrame(index=[updateidx], data = {'filetype': 'Ignition',
                                          'objectid': f'21231_Update_{upix}',
                                          'filepath': filepath,
                                          'datetime': gdfreference.loc[referenceidx, 'datetime'],
                                          'description': 'Maria2019',
                                          'geometry': Xgeom})

    usr.db.gdfignition = pd.concat([usr.db.gdfignition, gdfupdate])

    data['calculations'].append({'referenceidx': referenceidx,
                                  'igniteidx': igniteidx,
                                  'updateidx': updateidx,
                                  'Xt': Xt, 'X': X, 'Y': Y, 'aligned_geom': aligned_geom,
                                  'wdar': wdar, 'wsar': wsar})
    
data['gdfignition'] = usr.db.gdfignition

with open('/home/jovyan/data/output_05162023.pkl', 'wb') as f:
    pickle.dump(data, f)